In [ ]:
import torch.nn as nn

class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False):
        super(SeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, dilation, groups=in_channels, bias=bias)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, 1, 0, 1, 1, bias=bias)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class ResidualBlock(nn.Module):
    def __init__(self, in_channeld, out_channels):
        super(ResidualBlock, self).__init__()
        self.residual_conv = nn.Conv2d(in_channels=in_channeld, out_channels=out_channels, kernel_size=1, stride=2, bias=False)
        self.residual_bn = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        self.sepConv1 = SeparableConv2d(in_channels=in_channeld, out_channels=out_channels, kernel_size=3, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        self.relu = nn.ReLU()
        self.sepConv2 = SeparableConv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        self.maxp = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        res = self.residual_conv(x)
        res = self.residual_bn(res)
        x = self.sepConv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.sepConv2(x)
        x = self.bn2(x)
        x = self.maxp(x)
        return res + x

class Model(nn.Module):
    def __init__(self, num_classes):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(8, affine=True, momentum=0.99, eps=1e-3)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(8, momentum=0.99, eps=1e-3)
        self.relu2 = nn.ReLU()
        self.module1 = ResidualBlock(in_channeld=8, out_channels=16)
        self.module2 = ResidualBlock(in_channeld=16, out_channels=32)
        self.module3 = ResidualBlock(in_channeld=32, out_channels=64)
        self.module4 = ResidualBlock(in_channeld=64, out_channels=128)
        self.last_conv = nn.Conv2d(in_channels=128, out_channels=num_classes, kernel_size=3, padding=1)
        self.avgp = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, input):
        x = input
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.module1(x)
        x = self.module2(x)
        x = self.module3(x)
        x = self.module4(x)
        x = self.last_conv(x)
        x = self.avgp(x)
        x = x.view((x.shape[0], -1))
        return x

In [ ]:
!KAGGLE_USERNAME=dm33tri KAGGLE_KEY=efbaf0ad3856d5763312611da133d531 kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 79% 61.0M/77.3M [00:00<00:00, 107MB/s] 
100% 77.3M/77.3M [00:00<00:00, 131MB/s]
 87% 84.0M/96.6M [00:01<00:00, 72.0MB/s]
100% 96.6M/96.6M [00:01<00:00, 83.1MB/s]
  0% 0.00/7.01k [00:00<?, ?B/s]
100% 7.01k/7.01k [00:00<00:00, 6.44MB/s]
 92% 85.0M/92.0M [00:00<00:00, 84.3MB/s]
100% 92.0M/92.0M [00:01<00:00, 90.0MB/s]
 78% 15.0M/19.3M [00:00<00:00, 57.6MB/s]
100% 19.3M/19.3M [00:00<00:00, 75.6MB/s]


In [ ]:
!tar -xf /content/fer2013.tar.gz

In [ ]:
!ls /content/fer2013

fer2013.bib  fer2013.csv  README


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
import csv
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

if not torch.cuda.is_available():
    from torchsummary import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

shape = (44, 44)

class DataSetFactory:
    def __init__(self):
        images = []
        emotions = []
        private_images = []
        private_emotions = []
        public_images = []
        public_emotions = []
        with open('/content/fer2013/fer2013.csv', 'r') as csvin:
            data = csv.reader(csvin)
            next(data)
            for row in data:
                face = [int(pixel) for pixel in row[1].split()]
                face = np.asarray(face).reshape(48, 48)
                face = face.astype('uint8')
                if row[-1] == 'Training':
                    emotions.append(int(row[0]))
                    images.append(Image.fromarray(face))
                elif row[-1] == "PrivateTest":
                    private_emotions.append(int(row[0]))
                    private_images.append(Image.fromarray(face))
                elif row[-1] == "PublicTest":
                    public_emotions.append(int(row[0]))
                    public_images.append(Image.fromarray(face))
        print('training size %d : private val size %d : public val size %d' % (
            len(images), len(private_images), len(public_images)))
        train_transform = transforms.Compose([
            transforms.RandomCrop(shape[0]),
            transforms.RandomHorizontalFlip(),
            ToTensor(),
        ])
        val_transform = transforms.Compose([
            transforms.CenterCrop(shape[0]),
            ToTensor(),
        ])
        self.training = DataSet(transform=train_transform, images=images, emotions=emotions)
        self.private = DataSet(transform=val_transform, images=private_images, emotions=private_emotions)
        self.public = DataSet(transform=val_transform, images=public_images, emotions=public_emotions)

class DataSet(torch.utils.data.Dataset):
    def __init__(self, transform=None, images=None, emotions=None):
        self.transform = transform
        self.images = images
        self.emotions = emotions

    def __getitem__(self, index):
        image = self.images[index]
        emotion = self.emotions[index]
        if self.transform is not None:
            image = self.transform(image)
        return image, emotion

    def __len__(self):
        return len(self.images)

batch_size = 128
lr = 0.01
epochs = 300
learning_rate_decay_start = 80
learning_rate_decay_every = 5
learning_rate_decay_rate = 0.9
classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
network = Model(num_classes=len(classes)).to(device)

if not torch.cuda.is_available():
    summary(network, (1, shape[0], shape[1]))

optimizer = torch.optim.SGD(network.parameters(), lr=lr, momentum=0.9, weight_decay=5e-3)
criterion = nn.CrossEntropyLoss()
factory = DataSetFactory()
training_loader = DataLoader(factory.training, batch_size=batch_size, shuffle=True, num_workers=1)

validation_loader = {
    'private': DataLoader(factory.private, batch_size=batch_size, shuffle=True, num_workers=1),
    'public': DataLoader(factory.public, batch_size=batch_size, shuffle=True, num_workers=1)
}

min_validation_loss = {
    'private': 10000,
    'public': 10000,
}

for epoch in range(epochs):
    network.train()
    total = 0
    correct = 0
    total_train_loss = 0
    if epoch > learning_rate_decay_start and learning_rate_decay_start >= 0:
        frac = (epoch - learning_rate_decay_start) // learning_rate_decay_every
        decay_factor = learning_rate_decay_rate ** frac
        current_lr = lr * decay_factor
        for group in optimizer.param_groups:
            group['lr'] = current_lr
    else:
        current_lr = lr
    print('learning_rate: %s' % str(current_lr))
    for i, (x_train, y_train) in enumerate(training_loader):
        optimizer.zero_grad()
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        y_predicted = network(x_train)
        loss = criterion(y_predicted, y_train)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(y_predicted.data, 1)
        total_train_loss += loss.data
        total += y_train.size(0)
        correct += predicted.eq(y_train.data).sum()
    accuracy = 100. * float(correct) / total
    print('Epoch [%d/%d] Training Loss: %.4f, Accuracy: %.4f' % (
        epoch + 1, epochs, total_train_loss / (i + 1), accuracy))
    network.eval()
    with torch.no_grad():
        for name in ['private', 'public']:
            total = 0
            correct = 0
            total_validation_loss = 0
            for j, (x_val, y_val) in enumerate(validation_loader[name]):
                x_val = x_val.to(device)
                y_val = y_val.to(device)
                y_val_predicted = network(x_val)
                val_loss = criterion(y_val_predicted, y_val)
                _, predicted = torch.max(y_val_predicted.data, 1)
                total_validation_loss += val_loss.data
                total += y_val.size(0)
                correct += predicted.eq(y_val.data).sum()
            accuracy = 100. * float(correct) / total
            if total_validation_loss <= min_validation_loss[name]:
                min_validation_loss[name] = total_validation_loss
            print('Epoch [%d/%d] %s validation Loss: %.4f, Accuracy: %.4f' % (
                epoch + 1, epochs, name, total_validation_loss / (j + 1), accuracy))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 42, 42]              72
       BatchNorm2d-2            [-1, 8, 42, 42]              16
              ReLU-3            [-1, 8, 42, 42]               0
            Conv2d-4            [-1, 8, 40, 40]             576
       BatchNorm2d-5            [-1, 8, 40, 40]              16
              ReLU-6            [-1, 8, 40, 40]               0
            Conv2d-7           [-1, 16, 20, 20]             128
       BatchNorm2d-8           [-1, 16, 20, 20]              32
            Conv2d-9            [-1, 8, 40, 40]              72
           Conv2d-10           [-1, 16, 40, 40]             128
  SeparableConv2d-11           [-1, 16, 40, 40]               0
      BatchNorm2d-12           [-1, 16, 40, 40]              32
             ReLU-13           [-1, 16, 40, 40]               0
           Conv2d-14           [-1, 16,

FileNotFoundError: ignored

In [ ]:
network.eval()

Model(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(8, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(8, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (module1): ResidualBlock(
    (residual_conv): Conv2d(8, 16, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (residual_bn): BatchNorm2d(16, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (sepConv1): SeparableConv2d(
      (depthwise): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
      (pointwise): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(16, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (relu): ReLU()
    (sepConv2): SeparableConv2d(
      (depthwise): Conv2d(16, 16, kernel_size=(3, 3), strid

In [ ]:
from torch.utils.mobile_optimizer import optimize_for_mobile

traced = torch.jit.trace(network, torch.rand(1, 1, 44, 44))
optimize_for_mobile(traced)
traced.save("model.pt")